In [1]:
!pip install datasets transformers tensorflow numpy matplotlib opencv-python scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00


In [2]:
import numpy as np
from PIL import Image
from datasets import load_dataset

# تحميل مجموعة البيانات
dataset = load_dataset("yasimed/lung_cancer")

# دالة استخراج التصنيفات من النصوص
def extract_label(text):
    return 1 if "cancer" in text.lower() else 0  # تحويل النص إلى تصنيف رقمي

# استخراج الصور والتصنيفات مع التأكد من الاتساق
images = []
labels = []

for item in dataset['train']:
    # تحويل الصورة إلى كائن PIL
    image = item['image'].convert("RGB")  # تحويل الصورة إلى 3 قنوات (RGB)

    # تغيير الحجم إلى 224x224
    image = image.resize((224, 224))

    # تحويل الصورة إلى مصفوفة NumPy وتطبيع القيم بين 0 و 1
    image = np.array(image, dtype=np.float32) / 255.0

    images.append(image)
    labels.append(extract_label(item['text']))  # استخراج التصنيف من النص

# تحويل القوائم إلى مصفوفات NumPy بعد التأكد من التجانس
images = np.array(images, dtype=np.float32)
labels = np.array(labels, dtype=np.int32)

# التحقق من الأشكال
print("✅ شكل الصور:", images.shape)  # يجب أن يكون (613, 224, 224, 3)
print("✅ شكل التصنيفات:", labels.shape)  # يجب أن يكون (613,)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/313 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/59.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/613 [00:00<?, ? examples/s]

✅ شكل الصور: (613, 224, 224, 3)
✅ شكل التصنيفات: (613,)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


In [5]:
from sklearn.model_selection import train_test_split

# تقسيم البيانات إلى تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# طباعة حجم البيانات بعد التقسيم
print("✅ عدد صور التدريب:", X_train.shape[0])
print("✅ عدد صور الاختبار:", X_test.shape[0])


✅ عدد صور التدريب: 490
✅ عدد صور الاختبار: 123


In [6]:
# إنشاء محول بيانات التدريب
train_datagen = ImageDataGenerator(
    rotation_range=20,  # تدوير عشوائي حتى 20 درجة
    width_shift_range=0.1,  # إزاحة أفقية بنسبة 10%
    height_shift_range=0.1,  # إزاحة رأسية بنسبة 10%
    horizontal_flip=True,  # قلب الصورة أفقيًا
)

# تجهيز البيانات للتدريب والاختبار
train_generator = train_datagen.flow(X_train, y_train, batch_size=32, shuffle=True)
test_generator = ImageDataGenerator().flow(X_test, y_test, batch_size=32, shuffle=False)


In [7]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# تحميل EfficientNetB0 بدون الطبقات العليا
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# تجميد الطبقات الأصلية
base_model.trainable = False

# إضافة الطبقات المخصصة
x = GlobalAveragePooling2D()(base_model.output)  # تقليل الأبعاد
x = Dense(256, activation='relu')(x)  # طبقة مخفية مع تنشيط ReLU
x = Dropout(0.5)(x)  # تقليل فرط التكيف (Overfitting)
output = Dense(1, activation='sigmoid')(x)  # مخرجات ثنائية (سرطان أو لا)

# إنشاء النموذج
model = Model(inputs=base_model.input, outputs=output)

# تجميع النموذج
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# عرض ملخص النموذج
model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 224, 224, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 224, 224, 3)    │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 225, 225, 3)    │              0 │ rescaling_1[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 32)   │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 32)   │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 32)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 112, 112, 32)   │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 112, 112, 32)   │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 112, 112, 32)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand    

 Total params: 4,377,764 (16.70 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
# تدريب النموذج
history = model.fit(train_generator, validation_data=test_generator, epochs=40)

# حفظ النموذج بعد التدريب
model.save("lung_cancer_EfficientNET.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.6562 - loss: 0.6605 - val_accuracy: 0.7805 - val_loss: 0.5293
Epoch 2/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 312ms/step - accuracy: 0.7363 - loss: 0.5860 - val_accuracy: 0.7805 - val_loss: 0.5241
Epoch 3/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 394ms/step - accuracy: 0.7589 - loss: 0.5594 - val_accuracy: 0.7805 - val_loss: 0.5283
Epoch 4/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 304ms/step - accuracy: 0.7782 - loss: 0.5331 - val_accuracy: 0.7805 - val_loss: 0.5272
Epoch 5/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 333ms/step - accuracy: 0.7577 - loss: 0.5588 - val_accuracy: 0.7805 - val_loss: 0.5269
Epoch 6/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 303ms/step - accuracy: 0.7779 - loss: 0.5400 - val_accuracy: 0.7805 - val_loss: 0.5484
Epoch 7/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 393ms/step - accuracy: 0.7488 - loss: 0.5825 - val_accuracy: 0.7805 - val_loss: 0.5238
Epoch 8/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 303ms/step - accuracy: 0.7632 - loss: 0.5751 - val_accuracy: 0.7

In [9]:
# تقييم النموذج
loss, accuracy = model.evaluate(test_generator)
print(f"✅ دقة النموذج على بيانات الاختبار: {accuracy * 100:.2f}%")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.7882 - loss: 0.5133
✅ دقة النموذج على بيانات الاختبار: 78.05%


In [10]:
import numpy as np
from PIL import Image

def preprocess_image(image_path):
    """ تحميل الصورة وتحويلها إلى الشكل المناسب للنموذج """
    image = Image.open(image_path).convert("RGB")  # تحويل إلى 3 قنوات (RGB)
    image = image.resize((224, 224))  # تغيير الحجم إلى 224x224
    image = np.array(image, dtype=np.float32) / 255.0  # تطبيع القيم بين 0 و 1
    image = np.expand_dims(image, axis=0)  # إضافة بعد جديد ليتناسب مع الإدخال المتوقع (batch_size, 224, 224, 3)
    return image

# تجربة الصورة
image_path = "/content/image (5).png"  # ضع مسار الصورة هنا
processed_image = preprocess_image(image_path)

# التنبؤ باستخدام النموذج
prediction = model.predict(processed_image)

# تفسير النتيجة بناءً على النص الموجود في البيانات
def interpret_prediction(pred):
    """ تحويل التنبؤ الرقمي إلى نص بشري """
    if pred >= 0.5:
        return "مصاب بسرطان الرئة 🩸"
    else:
        return "غير مصاب بالسرطان ✅"

# عرض النتيجة النصية مع احتمالية الإصابة
result = interpret_prediction(prediction[0][0])
print(f"🔍 النتيجة: {result} (احتمال الإصابة: {prediction[0][0] * 100:.2f}%)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
🔍 النتيجة: مصاب بسرطان الرئة 🩸 (احتمال الإصابة: 77.27%)
